In [1]:
import torch
from utils import prepare_data
from train import get_dataloaders
import pandas as pd
import numpy as np
from params import PARAMS
from sklearn.model_selection import train_test_split
from main import init_model_params

from train import train_model_2
from main import model_predict, print_classification_report

RANDOM_SEED = 0
SEQ_LEN = 32
N_EPOCHS = 500
BATCH_SIZE = 32

# Load data

In [2]:
from utils import load_data



X_train, y_train, X_test, y_test = load_data('dataset/train/cross_subject_data_0_5_subjects.pickle')
print('Original shapes:')
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train = X_train[np.isin(y_train, [0, 1])]
y_train = y_train[np.isin(y_train, [0, 1])]
X_test = X_test[np.isin(y_test, [0, 1])]
y_test = y_test[np.isin(y_test, [0, 1])]

print('Shape with 2 classes (left, right arm):')
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train = np.vstack([X_train, X_test])
y_train = np.hstack([y_train, y_test])

X_train, y_train = prepare_data(X_train, y_train, SEQ_LEN)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=PARAMS['TEST_SIZE'], shuffle=True, random_state=RANDOM_SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=PARAMS['VALID_SIZE'], shuffle=True, random_state=RANDOM_SEED)
dataloaders = get_dataloaders(X_train, y_train, X_valid, y_valid, X_test, y_test, PARAMS['BATCH_SIZE'], random_seed=RANDOM_SEED, device=PARAMS['DEVICE'])

Original shapes:
X_train shape: (293612, 64)
y_train shape: (293612,)
X_test shape: (59040, 64)
y_test shape: (59040,)
Shape with 2 classes (left, right arm):
X_train shape: (146060, 64)
y_train shape: (146060,)
X_test shape: (29520, 64)
y_test shape: (29520,)


In [3]:
from layers import SelfAttentionLayer, BatchGraphConvolutionLayer
from torch import nn
import torch.nn.functional as F
import math


class GCRAMAuto(nn.Module):
    def __init__(self, seq_len, cnn_in_channels, cnn_n_kernels, cnn_kernel_size, cnn_stride, maxpool_kernel_size, maxpool_stride, lstm_hidden_size, is_bidirectional, lstm_n_layers, attn_embed_dim, n_classes, lstm_dropout_p, dropout1_p, dropout2_p, device):
        super(GCRAMAuto, self).__init__()

        self.dropout1_p = dropout1_p
        self.dropout2_p = dropout2_p

        self.gc1 = BatchGraphConvolutionLayer(seq_len, 1024, 64)
        self.gc2 = BatchGraphConvolutionLayer(1024, seq_len, 64)


        self.conv1 = nn.Conv2d(cnn_in_channels, cnn_n_kernels, kernel_size=cnn_kernel_size, stride=cnn_stride)
        self.maxpool1 = nn.MaxPool2d(kernel_size=maxpool_kernel_size, stride=maxpool_stride)

        cnn_output_size = (seq_len - cnn_kernel_size[1])//cnn_stride + 1
        maxpool_output_size = (cnn_output_size-maxpool_kernel_size[1])//maxpool_stride+1
        lstm_input_size = maxpool_output_size * cnn_in_channels * cnn_n_kernels
        self.lstm1 = nn.LSTM(input_size=lstm_input_size, hidden_size=lstm_hidden_size, batch_first=True, bidirectional=is_bidirectional, num_layers=lstm_n_layers, dropout=lstm_dropout_p)

        if is_bidirectional:
            self.attention = SelfAttentionLayer(hidden_size=lstm_hidden_size*2, attention_size=attn_embed_dim, return_alphas=True)
        else:
            self.attention = SelfAttentionLayer(hidden_size=lstm_hidden_size, attention_size=attn_embed_dim, return_alphas=True)
        
        self.flatten = nn.Flatten()

        if is_bidirectional:
            self.linear = nn.Linear(lstm_hidden_size*2, n_classes)
        else:
            self.linear = nn.Linear(lstm_hidden_size, n_classes)

        self.adj = nn.Parameter(torch.randn(64, 64), requires_grad=True)
        
    def forward(self, x):
        # print(x.size())
        # out = torch.einsum("ij,kjl->kil", self.adj, x)\
        out = self.gc1(x, self.adj)
        # print(out.size())
        out = self.gc2(out, self.adj)
        # print(out.size())

        out = out.unsqueeze(1)

        out = F.relu(self.conv1(out))
        out = self.maxpool1(out)
        
        out = self.flatten(out)
        out = out.unsqueeze(1)
        out = F.dropout(out, p=self.dropout1_p)

        out, (h_T, c_T) = self.lstm1(out)
        out = out[:, -1, :]

        out = out.unsqueeze(1)

        out, attn_weights = self.attention(out)
        out = F.dropout(out, p=self.dropout2_p)

        out = self.flatten(out)
        out = self.linear(out)

        return out

    def init_node_embeddings(self):
        stdv = 1. / math.sqrt(self.adj.size(1))
        self.adj.data.uniform_(-stdv, stdv)
        self.adj.data.fill_diagonal_(1)

In [4]:
model = GCRAMAuto(seq_len=SEQ_LEN, 
        cnn_in_channels=PARAMS['GCRAM_CNN_IN_CHANNELS'], 
        cnn_n_kernels=PARAMS['GCRAM_CNN_N_KERNELS'], 
        cnn_kernel_size=PARAMS['GCRAM_CNN_KERNEL_SIZE'], 
        cnn_stride=PARAMS['GCRAM_CNN_STRIDE'], 
        maxpool_kernel_size=PARAMS['GCRAM_MAXPOOL_KERNEL_SIZE'], 
        maxpool_stride=PARAMS['GCRAM_MAXPOOL_STRIDE'], 
        lstm_hidden_size=PARAMS['GCRAM_LSTM_HIDDEN_SIZE'], 
        is_bidirectional=PARAMS['GCRAM_LSTM_IS_BIDIRECTIONAL'], 
        lstm_n_layers=PARAMS['GCRAM_LSTM_N_LAYERS'], 
        attn_embed_dim=PARAMS['GCRAM_ATTN_EMBED_DIM'], 
        n_classes=PARAMS['N_CLASSES'], 
        lstm_dropout_p=PARAMS['GCRAM_LSTM_DROPOUT_P'], 
        dropout1_p=PARAMS['GCRAM_DROPOUT1_P'], 
        dropout2_p=PARAMS['GCRAM_DROPOUT2_P'], 
        device=PARAMS['DEVICE']).to(PARAMS['DEVICE'])

RuntimeError: Trying to create tensor with negative dimension -500: [256, -500]

In [ ]:
for input, label in dataloaders['train']:
    input = input.to(PARAMS['DEVICE'])
    label = label.to(PARAMS['DEVICE'])
    out = model(input)
    print(out.shape)
    break

torch.Size([64, 2])


In [ ]:
model = init_model_params(model, random_seed=RANDOM_SEED)
model.init_node_embeddings()

model = model.to(PARAMS['DEVICE'])

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=PARAMS['LR'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

# best_model, history = train_model(dataloaders, dataset_sizes, model, criterion, optimizer, scheduler, PARAMS['N_EPOCHS'], random_seed=random_seed)
best_model, history = train_model_2(model, optimizer, scheduler, criterion, dataloaders['train'], dataloaders['val'], N_EPOCHS, RANDOM_SEED, PARAMS['DEVICE'])
best_model = best_model.to(PARAMS['DEVICE'])

y_preds, y_test = model_predict(best_model, test_loader=dataloaders['test'])

cr, cm, auroc = print_classification_report(y_test, y_preds, PARAMS['N_CLASSES'], PARAMS['LABEL_MAP'])

train() called: model=GCRAMAuto, opt=Adam(lr=0.001000), epochs=500, device=cuda

Epoch   1/500, LR 0.0010, train loss: 0.7275, train acc: 0.5021, val loss: 0.6523, val acc: 0.4891
Epoch  10/500, LR 0.0010, train loss: 0.6951, train acc: 0.4976, val loss: 0.6380, val acc: 0.4781
Epoch  20/500, LR 0.0010, train loss: 0.6937, train acc: 0.5120, val loss: 0.6388, val acc: 0.4766
Epoch  30/500, LR 0.0009, train loss: 0.6939, train acc: 0.4923, val loss: 0.6406, val acc: 0.4641
Epoch  40/500, LR 0.0009, train loss: 0.6943, train acc: 0.5048, val loss: 0.6397, val acc: 0.4719
Epoch  50/500, LR 0.0008, train loss: 0.6931, train acc: 0.5054, val loss: 0.6366, val acc: 0.5078
Epoch  60/500, LR 0.0008, train loss: 0.6935, train acc: 0.5081, val loss: 0.6385, val acc: 0.4750
Epoch  70/500, LR 0.0007, train loss: 0.6937, train acc: 0.5089, val loss: 0.6378, val acc: 0.4781
Epoch  80/500, LR 0.0007, train loss: 0.6930, train acc: 0.5077, val loss: 0.6366, val acc: 0.4984
Epoch  90/500, LR 0.0007, tr

In [ ]:
print(cr)

{'0': {'precision': 0.4973958333333333, 'recall': 1.0, 'f1-score': 0.6643478260869565, 'support': 382}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 386}, 'accuracy': 0.4973958333333333, 'macro avg': {'precision': 0.24869791666666666, 'recall': 0.5, 'f1-score': 0.33217391304347826, 'support': 768}, 'weighted avg': {'precision': 0.24740261501736108, 'recall': 0.4973958333333333, 'f1-score': 0.33044384057971016, 'support': 768}}


In [ ]:
cm

array([[382,   0],
       [386,   0]])